In [1]:
%load_ext line_profiler

In [10]:
import pandas as pd
from math import lgamma, log, exp,fabs,inf
from scipy.special import comb
from statsmodels.stats.multitest import multipletests
import dask.dataframe as dd 
import numpy as np 

pd.options.display.float_format = '{:12.5e}'.format

def get_association(test,regdom): 
    """
    Function allowing from a file of genomic regions from CHIPseq 
    and a file of genomic regulatory domains to determine the names 
    of genes associated with at least one genomic region 

    Parameters
    ----------
    test : pd.dataFrame
        df of the tests pics => columns: ["Chr","Chr_Start","Chr_End"]
    
    regdom : pd.dataFrame
        df of the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

    Returns
    -------
    res : list
        list of gene associated with at least with one test peak
        
    Exemples 
    --------
    test = pd.DataFrame({
    ...    "Chr":["chr1"],
    ...    "Chr_Start":[1052028],
    ...    "Chr_End": [1052049]})

    regdom = pd.DataFrame({
    ...    "Chr":["chr1","chr1"],
    ...    "Chr_Start":[1034992,1079306],
    ...    "Chr_End": [1115089,1132016],
    ...    "Name":["RNF223","C1orf159"],
    ...    "tss":[1074306,1116089],
    ...    "strand":['-','-']})

    >>> get_association(test,regdom)        
        ['RNF223']
    
    """
    res = []
    for i in range(test.shape[0]):
        currTest = test.iloc[i]
        regdom_curr_test = regdom.loc[(regdom["Chr"] == currTest["Chr"])].sort_values("Chr_Start")
        regdom_curr_test = regdom_curr_test.loc[((regdom_curr_test["Chr_Start"] <= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] >= currTest["Chr_End"])) | ((regdom_curr_test["Chr_Start"] >= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] <= currTest["Chr_End"])) | ((regdom_curr_test["Chr_Start"] <= currTest["Chr_Start"]) & (regdom_curr_test["Chr_End"] >= currTest["Chr_End"]))] 
        res = res + list(regdom_curr_test["Name"])
    return list(dict.fromkeys(res))

def len_regdom(regdom:pd.DataFrame): 
    """
    Function to calculate for each gene name from regdom the
     size of the regulatory region for this gene in the genome 

    Parameters
    ----------    
    regdom : pd.dataFrame
        df of the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

    Returns
    -------
    dict
        dictionary in which each key corresponds to a gene name 
        from regdom and the value is the size of the regulatory 
        region for that gene
        
    Exemples 
    --------
    regdom = pd.DataFrame({
    ...    "Chr":["chr1","chr1"],
    ...    "Chr_Start":[1034992,1079306],
    ...    "Chr_End": [1115089,1132016],
    ...    "Name":["RNF223","C1orf159"],
    ...    "tss":[1074306,1116089],
    ...    "strand":['-','-']}))

    >>> len_regdom(regdom)
        {'RNF223': 80097, 'C1orf159': 52710}

    """
    test = regdom["Chr_End"]-regdom["Chr_Start"]
    return pd.DataFrame({"len":list(test)},index=regdom["Name"]).to_dict()["len"]

def number_of_hit(test,regdom): 
    """ 
    Function to calculate the number of hits from several 
    genomic regions and the file describing the regulatory regions

    Parameters
    ----------
    test : pd.dataFrame
        df of the tests pics => columns: ["Chr","Chr_Start","Chr_End"]
    
    regdom : pd.dataFrame
        df of the regulatory domains => columns: ["Chr"	"Chr_Start"	"Chr_End"	"Name"	"tss"	"strand"].

    Returns
    -------
    nb : int
        number of hit 
        
    Exemples 
    --------
    test = pd.DataFrame({
    ...    "Chr":["chr1"],
    ...    "Chr_Start":[1052028],
    ...    "Chr_End": [1052049]})

    regdom = pd.DataFrame({
    ...    "Chr":["chr1","chr1"],
    ...    "Chr_Start":[1034992,1079306],
    ...    "Chr_End": [1115089,1132016],
    ...    "Name":["RNF223","C1orf159"],
    ...    "tss":[1074306,1116089],
    ...    "strand":['-','-']})

    >>> number_of_hit(test,regdom)        
        1
    
    """
    nb=0
    regdom = regdom[["Chr","Chr_Start","Chr_End"]]
    for i in range(test.shape[0]): 
        chrom = test.iat[i,0]
        start = test.iat[i,1]
        end = test.iat[i,2]
        regdom_np = regdom["Chr"].to_numpy()
        reg_start = regdom["Chr_Start"].to_numpy()
        reg_end = regdom["Chr_End"].to_numpy()
        Chr_reduce = np.where(regdom_np == chrom)
        reg_start = np.take(reg_start,Chr_reduce,axis=0)[0]
        reg_end = np.take(reg_end,Chr_reduce,axis=0)[0]

        if any((reg_start <= start) & (reg_end >= end)):  
            nb += 1
    return nb

def betacf(a,b,x): 
    """ Used by betai: Evaluates continued fraction for incomplete beta function """
    maxit = 10000
    eps = 3.0e-7 
    fpmin = 1.0e-30
    qab = a+b
    qap = a+1
    qam = a-1
    c = 1
    d = 1-qab*x/qap
    if fabs(d) < fpmin :
        d = fpmin
    d = 1/d 
    h = d
    for m in range(1,maxit+1): 
        m2 = 2*m
        aa = m*(b-m)*x/((qam+m2)*(a+m2))
        d = 1.0+aa*d
        if (fabs(d) < fpmin) : 
            d = fpmin
        c = 1.0+aa/c
        if (fabs(c) < fpmin):
            c = fpmin
        d = 1.0/d
        h *= d*c
        aa = -(a+m)*(qab+m)*x/((a+m2)*(qap+m2))
        d = 1.0+aa*d  
        if (fabs(d) < fpmin):
            d = fpmin
        c = 1.0+aa/c
        if (fabs(c) < fpmin):
            c = fpmin
        d = 1.0/d
        dell = d*c
        h *= dell
        if (fabs(dell-1.0) < eps):
            break
    if (m > maxit):
        print("a or b too big, or MAXIT too small in betacf")
        return False
    return h

def betai(a,b,x):
    """Returns the incomplete beta function Ix(a, b)."""
    if x < 0 or x > 1 : 
        return False
    if x == 0 or x == 1 : 
        bt = 0.0
    else : 
        bt = exp(lgamma(a+b)-lgamma(a)-lgamma(b)+a*log(x)+b*log(1.0-x))
    if x < (a+1)/(a+b+2) : 
        return bt*betacf(a,b,x)/a
    return 1-bt*betacf(b,a,1-x)/b

def get_binom_pval(n:int,k:int,p:float) -> float:
    """
    This function allows to calculate the binomial probability 
    of obtaining k in a set of size n and whose probability is p 

    Parameters
    ----------
    n : int
        Number of genomic region in the test set 
    k : int 
        Number of test genomic regions in the regulatory domain of a gene with annotation
    p : float
        Percentage of genome annotated

    Returns
    -------
    float
        binomial probability
        
    Exemples 
    --------
    >>> get_binom_pval(100,2,0.2)
        0.9999999947037065
    
    """
    if k == 0 : return 1
    else : return betai(k,n-k+1,p)

def hypergeom_pmf(N, K, n, k):
    """
    Function to calculate the probability mass function for hypergeometric distribution

    Parameters
    ----------
    N : int
        Total number of gene in the genome
    K : int 
        Number of genes in the genome with annotation
    n : int
        Number of gene in the test set
    k : int
        Number of genes in the test gene set with annotation

    Returns
    -------
    float
        proability mass function
        
    Exemples 
    --------
    >>> hypergeom_pmf(100,10,30,1)
        0.11270773995748315
    
    """
    Achoosex = comb(K,k) if comb(K,k) != inf else 1e-308
    NAchoosenx = comb(N-K, n-k) if comb(N-K, n-k) != inf else 1e-308
    Nchoosen = comb(N,n) if comb(N,n) != inf else 1e-308
    return ((Achoosex)*NAchoosenx)/Nchoosen if Nchoosen > 1e-308 and (Achoosex)*NAchoosenx != 0.0 else 1e-308

def hypergeom_cdf(N, K, n, k):
    """
    Function to calculate the cumulative density funtion for hypergeometric distribution

    Parameters
    ----------
    N : int
        Total number of gene in the genome
    K : int 
        Number of genes in the genome with annotation
    n : int
        Number of gene in the test set
    k : int
        Number of genes in the test gene set with annotation

    Returns
    -------
    float
        Cumulative density function
        
    Exemples 
    --------
    >>> hypergeom_cdf(100,10,30,1)
        0.9770827595419788
    
    """
    return np.sum([hypergeom_pmf(N, K, n, x) for x in range(k,min(K,n)+1)])

In [3]:
def loader(test_data:None or str or pd.DataFrame,regdom_file:None or str or pd.DataFrame,chr_size_file:None or str or pd.DataFrame,annotation_file:None or str or pd.DataFrame):
    if type(regdom_file) == str:
        regdom = pd.read_csv(regdom_file,sep="\t",comment="#",
                    names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"object","tss":"int64","Strand":"object"})
    elif type(regdom_file) == pd.DataFrame:
        regdom = regdom_file.iloc[:,:6]
        colname = list(regdom.columns)
        try : 
            regdom = regdom.rename(columns={colname[0]:"Chr",colname[1]:"Chr_Start",colname[2]:"Chr_End",colname[3]:"Name",colname[4]:"tss",colname[5]:"Strand"})
        except :
            print("Error in the format of the regdom file")
            print("The regdom file must have the following columns : Chr, Chr_Start, Chr_End, Name, tss, Strand")
            return False 
    else : 
        regdom = regdom_file

    if type(test_data) == str : 
        test_data = pd.read_csv(test_data,sep="\t",comment="#",usecols=[0,1,2],
                        names=["Chr", "Chr_Start", "Chr_End"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64"})
    elif type(test_data) == pd.DataFrame : 
        test_data = test_data.iloc[:,:3]
        colname = list(test_data.columns)
        try : 
            test_data = test_data.rename(columns={colname[0]:"Chr",colname[1]:"Chr_Start",colname[2]:"Chr_End"})
        except : 
            print("Error in test dataframe, please check your input")
            print("Columns should be : chr...(type object), start(type int), end(type int)")
            return False
    else :
        pass

    
    if type(chr_size_file) == str :
        size = pd.read_csv(chr_size_file,sep="\t",comment="#",
                        names=["Chrom","Size"],dtype={"Chrom":"object", "Size":"int64"})
    elif type(chr_size_file) == pd.DataFrame :
        size = chr_size_file.iloc[:,:2]
        colname = list(size.columns)
        try : 
            size = size.rename(columns={colname[0]:"Chrom",colname[1]:"Size"})
        except : 
            print("Error in the format of the chr_size file")
            print("The chr_size file must have the following columns : Chrom, Size")
            return False
    else :
        size = chr_size_file

    if type(annotation_file) == str : 
        dask_df = dd.read_csv(annotation_file,sep=";",  comment = "#",
                        dtype={"ensembl":"object","id":"object","name":"object","ontology.group":"object","gene.name":"object","symbol":"object"},
                        usecols=["id","name","symbol"],low_memory=False)
        ann = dask_df.compute()
        ann = ann[ann['id'].str.match('^GO.*')== True]
    elif type(annotation_file) == pd.DataFrame : 
        ann = annotation_file.iloc[:,:4]
        colname = list(ann.columns)
        try : 
            ann = ann.rename(columns={colname[0]:"id",colname[1]:"name",colname[3]:"symbol"})
        except : 
            print("Error in the format of the annotation file")
            print("The annotation file must have the following columns : id, name, symbol")
            return False
    else :
        ann = annotation_file

    return test_data,regdom,size,ann


In [32]:
from numba import njit
import Cython

In [34]:
%load_ext cython 

In [41]:
%%cython
import numpy as np
import pandas as pd
def select_id(ann,id) : 
    id_np = ann["id"].to_numpy()
    name_np = ann["name"].to_numpy()
    symbol_np = ann["symbol"].to_numpy()
    idx_np = np.where(id_np == id)
    id_np = np.take(id_np,idx_np)[0]
    name_np = np.take(name_np,idx_np,axis=0)[0]
    symbol_np = np.take(symbol_np,idx_np,axis=0)[0]
    return pd.DataFrame({"id":list(id_np),"name":list(name_np),"symbol":list(symbol_np)})

def select_symbol(ann,symbol) : 
    id_np = ann["id"].to_numpy()
    name_np = ann["name"].to_numpy()
    symbol_np = ann["symbol"].to_numpy()
    idx_np = np.where(symbol_np == symbol)
    id_np = np.take(id_np,idx_np)[0]
    name_np = np.take(name_np,idx_np,axis=0)[0]
    symbol_np = np.take(symbol_np,idx_np,axis=0)[0]
    return pd.DataFrame({"id":list(id_np),"name":list(name_np),"symbol":list(symbol_np)})

In [45]:
%%cython
import numpy as np
import pandas as pd

def select_id(ann,id) : 
    id_np = ann["id"].to_numpy()
    name_np = ann["name"].to_numpy()
    symbol_np = ann["symbol"].to_numpy()
    idx_np = np.where(id_np == id)
    id_np = np.take(id_np,idx_np)[0]
    name_np = np.take(name_np,idx_np,axis=0)[0]
    symbol_np = np.take(symbol_np,idx_np,axis=0)[0]
    return pd.DataFrame({"id":list(id_np),"name":list(name_np),"symbol":list(symbol_np)})

def select_symbol(ann,symbol) : 
    id_np = ann["id"].to_numpy()
    name_np = ann["name"].to_numpy()
    symbol_np = ann["symbol"].to_numpy()
    idx_np = np.where(symbol_np == symbol)
    id_np = np.take(id_np,idx_np)[0]
    name_np = np.take(name_np,idx_np,axis=0)[0]
    symbol_np = np.take(symbol_np,idx_np,axis=0)[0]
    return pd.DataFrame({"id":list(id_np),"name":list(name_np),"symbol":list(symbol_np)})

def enrichment_binom_and_hypergeom(test,regdom,size,ann,asso) : 
    # Init Great
    res = {}
    hit={}

    # init Hypergeom
    hypergeom_gene_set=len(asso) # get the number of genes in the test gene set.
    hypergeom_total_number_gene = regdom.shape[0] #get the number of genes in the genome.

    # Init binom 
    n_binom = test.shape[0]# get the number of genomic region in the test set
    total_nu = size["Size"].sum()# get the total number of nucleotides in the genome

    ann_red = ann[ann["symbol"].isin(asso)]
    regdom = regdom[regdom["Name"].isin(list(ann[ann["id"].isin(list(ann_red["id"]))]["symbol"]))]#reduction of the regdom file by selecting only the genes whose GO ID is owned by a gene of the association 
    len_on_chr = len_regdom(regdom)# get the length of each regulatory domain 

    one = ann[:564427]
    two = ann[564428:1128854]
    three = ann[1128855:1693281]
    four = ann[1693282:]

    #Compute for all associating gene and for each GO id associated with the gene the probability. 
    for name in asso :
        # ann_name_gene = ann[ann["symbol"].isin([name])]
        tmp1 = select_symbol(one,name)
        tmp2 = select_symbol(two,name); tmp2 = tmp2.reset_index(drop=True)
        tmp3 = select_symbol(three,name); tmp3 = tmp3.reset_index(drop=True)
        tmp4 = select_symbol(four,name); tmp4 = tmp4.reset_index(drop=True)
        ann_name_gene = pd.concat([tmp1, tmp2,tmp3,tmp4],axis=0)

        id = ann_name_gene["id"]
        tmp = []
        for i in (list(id.unique())): 
            # gene_imply = ann[ann['id'].isin([i])]
            tmp1 = select_id(one,i)
            tmp2 = select_id(two,i); tmp2 = tmp2.reset_index(drop=True)
            tmp3 = select_id(three,i); tmp3 = tmp3.reset_index(drop=True)
            tmp4 = select_id(four,i); tmp4 = tmp4.reset_index(drop=True)
            gene_imply = pd.concat([tmp1, tmp2,tmp3,tmp4],axis=0)
            
            K_hypergeom = gene_imply.shape[0] # get be the number of genes in the genome with annotation
            curr_regdom = regdom.loc[regdom["Name"].isin(list(gene_imply["symbol"]))]
            k_hypergeom = curr_regdom.loc[curr_regdom["Name"].isin(asso)].shape[0] # get the number of genes in the test gene set with annotation

            if i not in list(hit.keys()) : 
                hit[i] = number_of_hit(test,curr_regdom)# get the number of test genomic regions in the regulatory domain of a gene with annotation
            k_binom=hit[i]
            nb_binom = sum([len_on_chr[i] for i in curr_regdom["Name"]])# get the portion of the genome in the regulatory domain of a gene with annotation
            tmp.append((k_binom,nb_binom,i,gene_imply.iloc[0]["name"],K_hypergeom,k_hypergeom))
        res.update({elem[2]:[ elem[3],get_binom_pval(n_binom,elem[0],elem[1]/total_nu), hypergeom_cdf(hypergeom_total_number_gene,elem[4],hypergeom_gene_set,elem[5]) ] for elem in tmp})
    return pd.DataFrame(res).transpose().rename(columns={0:"go_term",1:"binom_p_value",2:"hypergeom_p_value"}).replace(0,np.nan).sort_values(by="binom_p_value")

In [46]:
test = "../../data/tests/test_data/input/01_random.bed"
regdom = "../../data/human/hg38/regulatory_domain.bed"
size = "../../data/human/hg38/chr_size.bed"
t,r,s,a = loader(test,regdom,size,"../../../ontologies_total.csv")
asso=get_association(t,r)
test=t[:2]

In [47]:
%lprun -f enrichment_binom_and_hypergeom enrichment_binom_and_hypergeom(test,r,s,a,asso)

/home/icb/tom.mauger-birocheau/miniconda3/lib/python3.9/site-packages/line_profiler/ipython_extension.py:71: UserWarning: Could not extract a code object for the object <built-in function enrichment_binom_and_hypergeom>
  profile = LineProfiler(*funcs)


Timer unit: 1e-06 s

In [9]:
%lprun -f enrichment_binom_and_hypergeom enrichment_binom_and_hypergeom(test,r,s,a,asso)

Timer unit: 1e-06 s

Total time: 213.928 s
File: /tmp/ipykernel_11102/2426021515.py
Function: enrichment_binom_and_hypergeom at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def enrichment_binom_and_hypergeom(test,regdom,size,ann,asso) : 
     2                                               # Init Great
     3         1          4.0      4.0      0.0      res = {}
     4         1          3.0      3.0      0.0      hit={}
     5                                           
     6                                               # init Hypergeom
     7         1          4.0      4.0      0.0      hypergeom_gene_set=len(asso) # get the number of genes in the test gene set.
     8         1         26.0     26.0      0.0      hypergeom_total_number_gene = regdom.shape[0] #get the number of genes in the genome.
     9                                           
    10                                               # Init 